## Done

## ToDo
 - [ ] Create County-FacilityName => CCN/CMS Number
 - [ ] Manually Clean up some of the CMS Numbers
 - [ ] Create functions for some of the cells

## Done

In [1]:
import pandas as pd
import numpy as np
import json
import requests
import lxml.html as lh
import re
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.michigan.gov/coronavirus/0,9753,7-406-98163_98173-526911--,00.html'
url = 'https://www.michigan.gov/coronavirus/0,9753,7-406-98163_98173-526911--,00.html'

# Wayback Machine work!!!
#url = 'https://web.archive.org/web/20201130212042/https://www.michigan.gov/coronavirus/0,9753,7-406-98163_98173-526911--,00.html'
#url = 'https://web.archive.org/web/20200720171507/https://www.michigan.gov/coronavirus/0,9753,7-406-98163_98173-526911--,00.html'
# TODO - Wayback machine adds two extra tables at top we need to skip past
page = requests.get(url)
soup = BeautifulSoup(page.text, 'lxml')

In [3]:
tables = soup.find_all('table')

tmp = re.split('/| ', tables[0].find_all('caption')[0].get_text())
reporting_date = '-'.join(tmp[0:3])
print(reporting_date)

12-21-2020


In [4]:
# Create list of facilities
rows = tables[2].find_all('tr')
# Get List of fields from Header Row
fieldList = []
fields = rows[0].find_all('th')
for field in fields:
    fieldList.append(field.get_text())
#assert fieldList == ['FACILITY NAME', 'COUNTY','TYPE', 'NEW RESIDENT CASES', 'TOTAL RESIDENT CASES', 'NEW RESIDENT DEATHS', 'TOTAL RESIDENT DEATHS',
#                     'NEW STAFF CASES', 'TOTAL STAFF CASES', 'NEW STAFF DEATHS', 'TOTAL STAFF DEATHS']
num_facilities = 0
facilities = []
first_row = True
for row in rows:
    if first_row:
        first_row = False
        continue
    facility = {}
    tds = row.find_all('td')
    num_reported_vals = 0
    facility['ReportingDate'] = reporting_date
    for i in range(len(tds)):
        val = tds[i].get_text()
        if val == '--':
            val = float('Nan')
        else:
            num_reported_vals = num_reported_vals +1
        facility[fieldList[i]] = val   
    facilities.append(facility)
print(len(facilities))

1349


In [21]:
# facilities_df = pd.read_json(json.dumps(facilities), dtype={'ReportingDate': np.str, 'FACILITY NAME': str, 'COUNTY': str, 'TYPE': str,
#                                       'NEW RESIDENT CASES': float, 'TOTAL RESIDENT CASES': float, 'NEW RESIDENT DEATHS': float,
#                       'TOTAL RESIDENT DEATHS': float, 'NEW STAFF CASES': float, 'TOTAL STAFF CASES': float,
#                       'NEW STAFF DEATHS': float, 'TOTAL STAFF DEATHS': float})
facilities_df = pd.read_json(json.dumps(facilities), dtype=True)

In [22]:
facilities_df.to_csv('Reporting_data/MI_' + reporting_date + 'LTC_data.csv', index=False)

In [23]:
facilities_df.dtypes

ReportingDate            object
FACILITY NAME            object
COUNTY                   object
TYPE                     object
COMPLIANCE               object
NEW RESIDENT CASES       object
TOTAL RESIDENT CASES     object
NEW RESIDENT DEATHS      object
TOTAL RESIDENT DEATHS    object
NEW STAFF CASES          object
TOTAL STAFF CASES        object
NEW STAFF DEATHS         object
TOTAL STAFF DEATHS       object
dtype: object

In [24]:
facilities_df.columns

Index(['ReportingDate', 'FACILITY NAME', 'COUNTY', 'TYPE', 'COMPLIANCE',
       'NEW RESIDENT CASES', 'TOTAL RESIDENT CASES', 'NEW RESIDENT DEATHS',
       'TOTAL RESIDENT DEATHS', 'NEW STAFF CASES', 'TOTAL STAFF CASES',
       'NEW STAFF DEATHS', 'TOTAL STAFF DEATHS'],
      dtype='object')

In [28]:
facilities_df

,ReportingDate,FACILITY NAME,COUNTY,TYPE,COMPLIANCE,NEW RESIDENT CASES,TOTAL RESIDENT CASES,NEW RESIDENT DEATHS,TOTAL RESIDENT DEATHS,NEW STAFF CASES,TOTAL STAFF CASES,NEW STAFF DEATHS,TOTAL STAFF DEATHS
0,12-21-2020,Jamieson Nursing Home,ALCONA,SNF,,0,3,0,0,2,6,0,0
1,12-21-2020,Lincoln Haven Nursing & Rehab. Community,ALCONA,SNF,,0,28,1,3,0,18,0,0
2,12-21-2020,Medilodge of Munising,ALGER,SNF,,0,0,0,0,0,2,0,0
3,12-21-2020,Allegan County Medical Care Community,ALLEGAN,SNF,,1,1,0,0,1,9,0,0
4,12-21-2020,Beacon Home at The Oaks,ALLEGAN,AFC,,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1344,12-21-2020,Pleasant Lake Lodge North,WEXFORD,AFC,,0,0,0,0,0,0,0,0
1345,12-21-2020,Pleasant Lake Lodge South,WEXFORD,AFC,,0,0,0,0,0,0,0,0
1346,12-21-2020,Samaritas Senior Living Cadillac,WEXFORD,SNF,,15,20,0,0,7,16,0,0
1347,12-21-2020,Serenity Adult Foster Care,WEXFORD,AFC,,0,0,0,0,0,0,0,0


In [30]:
cols = ['NEW RESIDENT CASES', 'TOTAL RESIDENT CASES', 'NEW RESIDENT DEATHS',
       'TOTAL RESIDENT DEATHS', 'NEW STAFF CASES', 'TOTAL STAFF CASES',
       'NEW STAFF DEATHS', 'TOTAL STAFF DEATHS']
print(facilities_df['TYPE'].value_counts())
print(facilities_df[cols].head(10).astype(int).sum())
# print(facilities_df[cols].sum())

AFC    603
SNF    439
HFA    307
Name: TYPE, dtype: int64


ValueError: invalid literal for int() with base 10: '.'

In [ ]:
facilities_df.sort_values(by='TYPE', ascending=False).head(30)